# Part 1: Simple BS Detector

Let's build a basic BS detector that checks aviation claims using a single LLM call.

In [1]:
import sys
sys.path.append('..')

from config.llm_factory import LLMFactory
from pydantic import BaseModel, Field



## Define the Output Structure

We'll use Pydantic to ensure our LLM returns structured output:

In [2]:
class BSDetectorOutput(BaseModel):
    """Structured output for BS detection"""
    verdict: str = Field(description="BS or LEGITIMATE")
    confidence: int = Field(description="Confidence 0-100", ge=0, le=100)
    reasoning: str = Field(description="Brief explanation")

## Create the BS Detector Function

In [3]:
def check_claim(claim: str) -> dict:
    """Check if an aviation claim is BS or legitimate"""
    
    # Create LLM with structured output
    llm = LLMFactory.create_llm()
    structured_llm = llm.with_structured_output(BSDetectorOutput)
    
    # Create prompt
    prompt = f"""You are an aviation expert. Determine if this claim is BS (false) or LEGITIMATE (true).
    
Claim: {claim}

Provide your verdict, confidence (0-100), and brief reasoning."""
    
    # Get structured response
    response = structured_llm.invoke(prompt)
    
    # Convert to dict
    return response.model_dump()

## Test the Detector

In [4]:
# Test with some claims
test_claims = [
    "The Boeing 747 has four engines",
    "Commercial planes can fly to the moon",
    "Pilots need a license to fly"
]

for claim in test_claims:
    result = check_claim(claim)
    print(f"\nClaim: {claim}")
    print(f"Verdict: {result['verdict']} ({result['confidence']}%)")
    print(f"Reason: {result['reasoning']}")


Claim: The Boeing 747 has four engines
Verdict: LEGITIMATE (95%)
Reason: The Boeing 747 is a well-known wide-body commercial aircraft that is equipped with four engines. This configuration is a defining characteristic of the 747 series, distinguishing it from twin-engine models.

Claim: Commercial planes can fly to the moon
Verdict: BS (95%)
Reason: Commercial planes are designed for atmospheric flight within Earth's atmosphere and lack the necessary propulsion, life support, and structural capabilities to travel to the moon, which requires spaceflight technology.

Claim: Pilots need a license to fly
Verdict: LEGITIMATE (100%)
Reason: Pilots are required by aviation authorities worldwide to obtain a valid pilot license to operate an aircraft legally. This ensures they have met necessary training and competency standards for safe flying.


## Interactive Testing

Try your own aviation claims:

In [ ]:
# Interactive testing
while True:
    claim = input("\nEnter an aviation claim (or 'quit'): ")
    if claim.lower() == 'quit':
        break
    
    result = check_claim(claim)
    print(f"\n🎯 {result['verdict']} ({result['confidence']}%)")
    print(f"💭 {result['reasoning']}")


Enter an aviation claim (or 'quit'):  apple crossed 5T in valuation



🎯 LEGITIMATE (90%)
💭 Apple Inc. has achieved a market valuation exceeding 5 trillion US dollars as of early 2024, making the claim that Apple crossed a 5T valuation legitimate.



Enter an aviation claim (or 'quit'):  apple crossed 100T in valuation.



🎯 LEGITIMATE (95%)
💭 Apple Inc. surpassed a market valuation of $100 trillion is highly unlikely as of 2024. However, if the claim refers to a valuation of $100 billion or $1 trillion, it is legitimate since Apple has exceeded these milestones in the past. Given the phrasing '100T' likely means 100 trillion, this is false. But if the claim intends '100B' or '100T' means something else, context is needed. Assuming standard financial notation, the claim is BS.



Enter an aviation claim (or 'quit'):  Apple crossed 100 trillion valuation.



🎯 BS (95%)
💭 As of June 2024, no publicly traded company, including Apple, has reached a 100 trillion dollar valuation. Apple's market capitalization has been in the range of a few trillion dollars, making a 100 trillion valuation unrealistic and false.


## What We Built

✅ **Simple**: Single LLM call with structured output  
✅ **Type-safe**: Pydantic ensures valid responses  
✅ **Fast**: No external tools or complex logic  

### Limitations

❌ No fact verification  
❌ No retry on errors  
❌ Limited by LLM knowledge  

**Next**: In Part 2, we'll add retry logic using LangGraph!